<a href="https://colab.research.google.com/github/Spartan-119/A-B-Testing-Approach-for-Comparing-Performance-of-ML-Models/blob/main/a_b_testing_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing all the necessary packages
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 --upgrade
!pip install transformers --upgrade
!pip install tqdm
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.7 MB/s eta 0:00:00


In [2]:
# importing all the necessary libraries
import json
import os
import timeit
import collections
import time
from pprint import pprint
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, squad_convert_examples_to_features
from transformers.data.processors.squad import SquadV2Processor,SquadResult
from transformers.data.metrics.squad_metrics import (
    compute_predictions_log_probs,
    compute_predictions_logits,
    squad_evaluate,
)

In [3]:
DO_LOWER_CASE = True
NBEST_SIZE = 20
DOC_STRIDE = 128
MAX_SEQ_LENGTH = 384
MAX_QUERY_LENGTH = 64
MAX_ANSWER_LENGTH = 30
DATA_DIR = 'data/squad'
PREDICT_FILE = 'dev-v2.0.json'

BERT_MODEL_TYPE = 'bert'
BERT_MODEL_HF_PATH = 'twmkn9/bert-base-uncased-squad2'
BERT_OUTPUT_DIR = 'models/bert/twmkn9_bert-case-uncased-squad2'

DISTILBERT_MODEL_TYPE = 'distilbert'
DISTILBERT_MODEL_HF_PATH = 'twmkn9/distilbert-base-uncased-squad2'
DISTILBERT_OUTPUT_DIR = 'models/distilbert/twmkn9_distilbert-base-uncased-squad2'

# Downloading and Exploring the dataset

In [4]:
# downloading the dataset
!wget -P data/squad/ https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-08-29 16:23:59--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘data/squad/dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  --.-KB/s    in 0.09s   

2023-08-29 16:23:59 (47.8 MB/s) - ‘data/squad/dev-v2.0.json’ saved [4370528/4370528]



#### <i>Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

#### SQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering.</i>


## Loading the DEV set using Hugging Face's data processors

I am going to make use of [Processors](https://huggingface.co/transformers/main_classes/processors.html) to facilitate basic processing tasks with some canonical NLP datasets. The processors can be used for loading datasets and converting their examples to features for direct use in the model. More specifically, we will be using the [SQuAD processors](https://huggingface.co/transformers/main_classes/processors.html#squad)

In [5]:
def to_list(tensor):
  return tensor.detach().cpu().tolist()

In [6]:
def load_and_cache_examples(model_name_or_path,
                            data_dir= DATA_DIR,
                            predict_file=PREDICT_FILE,
                            max_seq_length=MAX_SEQ_LENGTH,
                            doc_stride=DOC_STRIDE,
                            max_query_length=MAX_QUERY_LENGTH,
                            overwrite_cache=True):

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False)
    # Load data features from cache or dataset file
    input_dir = data_dir if data_dir else "."
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}_{}_{}".format(
            "dev",
            list(filter(None, model_name_or_path.split("/"))).pop(),
            str(max_seq_length),
        ),
    )

    # Init features and dataset from cache if it exists
    if os.path.exists(cached_features_file) and not overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else:

        processor = SquadV2Processor()

        examples = processor.get_dev_examples(data_dir, filename=predict_file)

        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset="pt",
            threads=1,
        )


    return dataset, examples, features

In [7]:
dataset, examples, features = load_and_cache_examples(BERT_MODEL_HF_PATH)

add example index and unique id: 100%|██████████| 11873/11873 [00:00<00:00, 573588.71it/s]


In [8]:
print(f'There are {len(examples)} examples in the dev dataset.')

There are 11873 examples in the dev dataset.


This list of examples contains objects of type transformers.data.processors.squad.SquadExample.
We use the functionbelow to extract the information we want from such objects.
More specifically: 'qid', 'question_text', 'context_text' and 'answer'.

I will first create some extra variables to help on manipulation of data.

In [9]:
# generating some maps to help identify examples of interest.
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]

And also, the function below to help on extracting information given a `qid` (question unique identifier)

In [10]:
def display_example(qid: str) -> None:
  idx = qid_to_example_index[qid]
  q = examples[idx].question_text
  c = examples[idx].context_text
  a = [answer['text'] for answer in examples[idx].answers]

  print(f'Examples {idx} of {len(examples)}\n------------------')
  print(f'Q: {q}\n')
  print('Context:')
  pprint(c)
  print(f'\nTrue Answers:\n{a}')

## Positive Example

50% of the examples in the test set are questionst hat have answers contained within their corresponding passage. In these cases, up to 5 possible correct answers are provided. Such answers must come directly from the passage, we will see later, however, that there are several ways to arrive at a "correct" answer.

In [11]:
display_example(answer_qids[2456])

Examples 4959 of 11873
------------------
Q: It is now possible to convert old relative ages into what type of ages using isotopic dating?

Context:
('At the beginning of the 20th century, important advancement in geological '
 'science was facilitated by the ability to obtain accurate absolute dates to '
 'geologic events using radioactive isotopes and other methods. This changed '
 'the understanding of geologic time. Previously, geologists could only use '
 'fossils and stratigraphic correlation to date sections of rock relative to '
 'one another. With isotopic dates it became possible to assign absolute ages '
 'to rock units, and these absolute dates could be applied to fossil sequences '
 'in which there was datable material, converting the old relative ages into '
 'new absolute ages.')

True Answers:
['absolute ages', 'rock units', 'new absolute']


## Negative Example

The remaining 50% of the questions in the test set do not have an answer. This is important as in a real life Q&A system, the model needs to learn when **NOT TO ANSWER.**

In [12]:
display_example(no_answer_qids[1235])

Examples 2520 of 11873
------------------
Q: What is difficult with a satellite-to-noise ratio?

Context:
('Oxygen presents two spectrophotometric absorption bands peaking at the '
 'wavelengths 687 and 760 nm. Some remote sensing scientists have proposed '
 'using the measurement of the radiance coming from vegetation canopies in '
 'those bands to characterize plant health status from a satellite platform. '
 'This approach exploits the fact that in those bands it is possible to '
 "discriminate the vegetation's reflectance from its fluorescence, which is "
 'much weaker. The measurement is technically difficult owing to the low '
 'signal-to-noise ratio and the physical structure of vegetation; but it has '
 'been proposed as a possible method of monitoring the carbon cycle from '
 'satellites on a global scale.')

True Answers:
[]


## Metrics for Q&A Systems

When measuring the performance of a machine learning system, we need to think about both **model** and **customer metrics.**

Q&A systems are usually measured by two dominant metrics: **F1** and **Exact Match (EM)**. They are computed on individual question and answer **pairs**. When multiple correct answers are available for a given question the maximum score over all possible correct answers is computed. Overall EM and F1 scores are computed for a model by averaging over the individual example scores.

### Exact Match:

For each Q&A pair, if the characters of the model's prediction are an exact matvh of the characters of any of the True Answer(s), **EM = 1**, else, **EM = 0**. This is a strict all-or-nothing metric, which may have little value for final customers of a Q&A system. It may be beneficial only when assessing against a negative example; if the model predicts any text at all, it automatically receives a 0 for that example.

### F1 Score:

Almost all classification problems rely on F1 score to measure the model performance. It is mostly appropriate when we care equally about precision and recall. On an QnA system, however, it is computed over the individual words in the prediction against those in the True answer. The number of shared words between the prediction and the trust is the basis of F1 score. While **precision** is the ratio between the number of shared words to the total number of words in the prediction; **Recall** is the ratio of the number of shared words to the toal number of words in the ground truth.

### Latency

Latency is an important metric for ML systems. in the QnA example, it is of the utmost importance when the system is used in a conversational application. <br> For Example, Alexa and Google home are devices that have very strict latency constraints as the users expect an answer within a few seconds or macroseconds after the question is asked. When updating models we should take this dimension according to the application of the system.

### Answer Rate

In the QnA systems, models that attempt to answer every question are often perceived inaccurate. The system should only provide an output when it is confident enough to do so. In other words, when the probabiliries of predictions are above a certain threshold. <br> In some applications, a model should be able to say *I don't know* or *The context has not enough information to answer the question.*

# Q&A Models

Question and Answering make use of Large Language Models (LLMs) as any other classification problem in NLP. The main difference relies on how the input and the output is provided to the model. Generally speaking models are trained to match the true answer to the question as they are provided together as an input to the model.

# Bert

BERT is a neural approach to pre-train language representations which obtains near SOTA results on a wide array of NLP tasks, including SQuAD Question Answering dataset <br><br>

Developed in 2019, BERT achieves 80.422% in the EM score and 83.118% in the F1 score.<br><br>

BERT-base has 110 million parameters and BERT-large has 340 million parameters.

## Model Parameters comparison

As LLMs were developed, the amount of parameters in these models have grown exponentially. Although this improves model performance, it comes at a cost: Latency. As it will be discussed, for use cases where inference is done on batches that may have less impact, however, on real time systems such as voice assistants or web search, latency plays a major role on deciding whether one model is better than the other.

# BERT Input

[CLS] context [SEP] question [SEP] [PAD] [PAD] [PAD]

**context** = "The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations."

**question** = "What organization is the IPCC a part of?"

**after being merged by the tokenizer**:
```
"[CLS] The Intergovernmental Panel on Climate Change (IPCC) is a scientific intergovernmental body under the auspices of the United Nations. [SEP] What organization is the IPCC a part of? [SEP] [PAD] [PAD] [PAD]"
```


**token-id format**:

[101, 1109, 11300, 2758, 24472, 15595, 20339, 1113, 13540, 9091, 113, 14274, 12096, 114, 1110, 170, 3812,
 9455, 2758, 24472, 15595, 1404, 1223, 1103, 22105, 1104, 1103, 1244, 3854, 119, 102, 1327, 2369, 1110, 1103,
 14274, 12096, 170, 1226, 1104, 136, 102, 0, 0, 0]

# Loading Pre-Trained BERT from Hugging Face's repository.

In [13]:
tokenized = AutoTokenizer.from_pretrained(BERT_MODEL_HF_PATH, use_fast = False)
model = AutoModelForQuestionAnswering.from_pretrained(BERT_MODEL_HF_PATH)

Some weights of the model checkpoint at twmkn9/bert-base-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Utility Functions

Given a question_id, model and tokenizer, we get an answer text. In here we get the maximum probability of beginning and end for the answer in the softmax output.

In [14]:
def get_prediction(qid: str, model: AutoModelForQuestionAnswering, tokenizer: AutoTokenizer):
  # given a question_id (qas_id or qid), load the example, get the model outputs and generate an answer
  question = examples[qid_to_example_index[qid]].question_text
  context = examples[qid_to_example_index[qid]].context_text

  inputs = tokenizer.encode_plus(question, context, return_tensors = 'pt')

  outputs = model(**inputs)

  answer_start = torch.argmax(outputs[0])       # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start: answer_end]))

  return answer

I am creatinga simple function that given an example list, it extracts the gold answers.

In [15]:
def get_gold_answers(example):
  """
  helper function that retrieves all possible true answers from a squad2.0 example.
  """

  gold_answers = [answer['text'] for answer in example.answers if answer['text']]

  # if gold_answers doesn't exist, it's because this is a negative example.
  # the only correct answer is an empty string then in this case.
  if not gold_answers:
    gold_answers = [""]

  return gold_answers

For metrics like EM, we need to make sure that texts are normalized so we can compare on a character level.

In [16]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s: str):
  """
  removing articles and punctuation, and standardizing whitespace are all typical text processing steps.
  """
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.puncutation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))